### About this Version

After having talked to Herbert und Teresa (12.07.22) we have decided on a new-ish approach on how to tackle the problem with the primitive vector in 2d. This notebook copies part of the old versions, but is not a copy in itself, so that hopefully the result is a bit cleaner.

In [1]:
import random as rd
import numpy as np
import numpy.linalg as la
#import math
from math import gcd
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D

import sympy as sp

In [2]:
def gcd3(a,b,c):
    return gcd(gcd(a,b),c)

def gcdExtended(a, b): 
    # Base Case 
    if a == 0 :  
        return b,0,1

    gcd,x1,y1 = gcdExtended(b%a, a) 

    # Update x and y using results of recursive 
    # call 
    x = y1 - (b//a) * x1 
    y = x1 

    return gcd,x,y

def lcm(a, b):
    return abs(a*b) // gcd(a, b)


In [3]:
def gen_int_basis(d=3, a=5):
    # d is the dimension of the ambient space and the number of resulting vectors
    # a is the maximum length of a resulting vector in the sup norm
    basis = np.zeros((d,d),dtype=np.int32)
    
    det = la.det(basis)
    counter = 0
    
    
    while abs(det) < 1:
        for i in range(d):
            basis[:,i] = gen_int_vec(d,a) 
        
        det = la.det(basis)
        counter += 1
        if counter == 100:
            raise RuntimeError('Could not generate basis.')
            break 
            
    return basis

def gen_int_vec(d=3, a=5):
    vec = np.zeros(d,dtype=np.int32)
    for i in range(d):
        vec[i] = rd.randint(-a,a)
    if False not in (vec == 0):
        vec = gen_int_vec(d,a)
        print("recalc gen_int_vec")
    return vec

### Lemma 3.1 (Parallel Vectors). 
Let $Z \subset \mathbb Z^d$ be a sublattice with $dim Z = d$. Then for every
 $u \in \mathbb Z^d$, there exists a vector $v \in Z$ that is parallel to $u$.

In [4]:
# function that takes basis (which generates sublattice Z) and vector v
# outputs vector u as linear combination of basis elements, which is parallel to v
# also outputs or at least explicitely calculates the coefficients involved

In [6]:
def L31_parallel_vectors(basis,u):
    # function that takes basis (which generates sublattice Z) and vector v
    # outputs vector u as linear combination of basis elements, which is parallel to v
    # also outputs or at least explicitely calculates the coefficients involved
    # input is numpy arrays, output is sympy matrices
    G = basis
    Gi = G**(-1)
    denom_list = []
    p = 1
    for entry in Gi:
        if entry.q not in denom_list:
            denom_list.append(entry.q)
            p *= entry.q
    
    up = p*Gi*u
    v = G*up
    if len(u)==3:
        v = v / gcd3(up[0],up[1],up[2])
        coeff = up / gcd3(up[0],up[1],up[2]) #dividng by gcd of coefficients gives us the shortest parallel vector in the span of the original basis
    if len(u)==2:
        v = v / gcd(up[0],up[1])
        coeff = up / gcd(up[0],up[1])
    
    # to find shortest vector in the span of the new basis (four vectors), 
    # we find out the ratios of the two vectors 
    # get them to integer by multiplying with denominators
    # calculate the gcd of the resulting numbers
    # then divide the gcd (or the vector associated to it) by the denominator again
    for i in range(len(u)):
        if u[i]!=0:
            num1 = v[i]
            num2 = u[i]
            denom = num1.q*num2.q
            new_gcd, a, b = gcdExtended(num1*denom,num2*denom)
            x = new_gcd / sp.sympify(num1.p*num2.q)
            v = v/num1*new_gcd
            coeff = coeff * a
            u_coeff = b
            break
    
    return v, coeff, u_coeff

In [7]:
# Next we perform orthogonal projection along v4


def project(projector, projectee):
    return projectee - projectee.dot(projector)/projector.dot(projector) * projector

def proj_3d_basis(basis,u):
    Pv1 = project(u, basis[:,0])
    Pv2 = project(u, basis[:,1])
    Pv3 = project(u, basis[:,2])
    Pv = sp.Matrix([Pv1[:], Pv2[:], Pv3[:]]).T
    return Pv

def proj_2d_basis(basis,u):
    Pv1 = project(u,basis[:,0])
    Pv2 = project(u,basis[:,1])
    Pv = sp.Matrix([Pv1[:], Pv2[:]]).T
    return Pv
    
    

In [43]:

#def skip_coord(vec,n=1):
#    """
#    Takes in 3-component vector and returns corresponding 2-component vector
#    given by removing the nth coordinate (n = 1,2,3).
#    """
#    ind=n-1
#    if n == 1:
#        new_vec = [vec[ind+1],vec[ind+2]]
#    elif n == 2:
#        new_vec = [vec[ind-1],vec[ind+1]]
#    elif n == 3:
#        new_vec = [vec[ind-2],vec[ind-1]]
#    # Note that this not very elegant approach is because I don't want the coordinates to be permuted cyclically,
#    # but instead just want to skip the given entry
#    
#    return sp.Matrix(new_vec)




#
#def skip_corr_coord(Pv, u):
#    """
#    Takes in 
#    - 3x3 sympy matrix Pv of vectors projected orthogonally along u
#    - projection vector u.
#    
#    Returns
#    - 2x3 sympy matrix containing original vectors of Pv, 
#    but with one coordinate removed. 
#    
#    Note:
#    In general, removed coordinate is first one (index 0).
#    If u is orthogonal to direction of removed coordinate, 
#    this operation would create 1-dim vectors, not 2-dim,
#    thus losing information. Depending on u another coorindate
#    might thus be skipped.
#    """
#    e1 = sp.Matrix([1,0,0])
#    e2 = sp.Matrix([0,1,0])
#    e3 = sp.Matrix([0,0,1])
#    """
#    !!!!!!!
#    These scalar products are just the first, second and third coordinates
#    We could also take the largest coordinate of u and take that??
#    !!!!!!!!!!!!!
#    """
#    if u.dot(e1) == 0: # u is orthogonal to e1
#        if u.dot(e2) == 0: # u is orthgonal to e2
#            #project onto the e1, e2 plane
#            Pv1_2 = skip_coord(Pv[:,0],3) # the subscript 2 stands for TWO coordinates
#            Pv2_2 = skip_coord(Pv[:,1],3)
#            Pv3_2 = skip_coord(Pv[:,2],3)
#            pass
#        else:
#            #project the Pvi onto the e1, e3 plane
#            Pv1_2 = skip_coord(Pv[:,0],2)
#            Pv2_2 = skip_coord(Pv[:,1],2)
#            Pv3_2 = skip_coord(Pv[:,2],2)
#            pass
#    else:
#        #project the Pvi onto e2, e3 plane
#        Pv1_2 = skip_coord(Pv[:,0],1)
#        Pv2_2 = skip_coord(Pv[:,1],1)
#        Pv3_2 = skip_coord(Pv[:,2],1)
#        pass
#    
#    return sp.Matrix([Pv1_2[:],Pv2_2[:],Pv3_2[:]]).T


    
    
def skip_corr_coord(Pv, u):
    ind = np.argmax(np.abs(u))
    Pv1_2 = skip_coord(Pv[:,0],ind)
    Pv2_2 = skip_coord(Pv[:,1],ind)
    Pv3_2 = skip_coord(Pv[:,2],ind)
    
    return sp.Matrix([Pv1_2[:],Pv2_2[:],Pv3_2[:]]).T



def skip_coord(vec,n=1):
    """
    Takes in 3-component vector and returns corresponding 2-component vector
    given by removing the nth coordinate (n = 0,1,2).
    """
    if n == 0:
        new_vec = [vec[1],vec[2]]
    elif n == 1:
        new_vec = [vec[0],vec[2]]
    elif n == 2:
        new_vec = [vec[0],vec[1]]
    
    return sp.Matrix(new_vec)



In [38]:
np.abs(u)

array([0, 1, 0], dtype=int32)

In [17]:
def lcm_mx(M):
        """
        Takes sympy 2x2 matrix with rational coefficients and 
        returns lcm of denominators of all entries
        """
        lcm_denom = M[0,0].q
        for entry in M[:]:
            lcm_denom = lcm(lcm_denom,entry.q)
        return lcm_denom

In [35]:

# now that we have our 2-dimensional vectors,
# we want to find a linearly dependent vector
# once we have that, we want to find the primitive, as we did before
def proj2induct_2d(Pv,u):
    """
    Takes 3x3 sympy matrix with vectors projected to orthogonal complement
    and returns 
    - a 2x2 sympy matrix (picks two linearly independant column vectors, skips one coordinate) 
    - 'additional' vector to perform next orth projection
    """
    Pv_2d = skip_corr_coord(Pv,u)
    
    # search for vector to make new "extra"
    for i in range(4): #this just remember the variable "i"
        ind = [(i+1)%3,(i+2)%3]
        if Pv_2d[:,ind].det() != 0:
            true_i=i
            break
        
        if i==3: # this is outside of range, meaning no subset is basis
            print("something went wrong! No linear ind. subset found.")
            print(Pv)
            print(Pv_2d)
            """
            !!!!!
            Put in Real Error Message
            """
    
    
    print(true_i)
    u_2d = Pv_2d[:,true_i]
    basis_2d = Pv_2d[:,ind]
    
    # determine lcm of denomiators
    lcm_denom = lcm_mx(Pv_2d)
    
    # making stuff integer
    v_2d, coeff_basis_2d, coeff_u_2d = L31_parallel_vectors(basis_2d*lcm_denom, u_2d*lcm_denom)
    v_2d /= lcm_denom
    

    # calculate what coefficients of v are in the order of the original basis,
    # not, as is outputted here, in the order of Pv_2d
    prim_coeff = [0,0,0,0]
    prim_coeff[ind[0]] = coeff_basis_2d[0]
    prim_coeff[ind[1]] = coeff_basis_2d[1]
    prim_coeff[true_i] = coeff_u_2d
    
    return basis_2d, u_2d, true_i, v_2d, prim_coeff
    


In [28]:
def nonzero_entry(vec):
    for i in range(len(vec)):
        if vec[i]!=0:
            break
    return i



def gcd_of_2dvecs(vec_mx):
    """
    Takes two 2d sympy vectors in form of a matrix and returns their gcd, 
    as well as the coefficients to build the gcd
    
    vec_mx       ... 2x2 matrix containing two column-vectors
    new_1d_basis ... gcd of vectors in vec_mx
    x, y         ... coefficients w.r.t. vectors in vec_mx
    """
    
    # look for a non-zero coordinate
    i = nonzero_entry(vec_mx[:,0])
    
    # take lcm of denominators of the i'th entries
    denoms = lcm_mx(vec_mx[i,:])

    a = vec_mx[i,0]*denoms
    b = vec_mx[i,1]*denoms
    c, x,y = gcdExtended(a,b)
    return x,y



In [20]:

# the steps from lemma 3.3 seem so unneccessary ...
def multiple_L_3_3(u,v):
    # following the steps in Lemma 3.3 (Smallest Common Superlattice)
    # again we need to make sure we are not looking at a 0 entry

    for i in range(len(u)):
        if u[i]!=0:
            break

    pq = v[i]/u[i]
    p = pq.p
    q = pq.q

    # replace u by u_new as in Lemma 3.3
    u_factor = sp.sympify(1)/q * gcd(p,q)
    v_factor = sp.sympify(1)/p * gcd(p,q)
    u_new = u_factor * u
    
    # new basis is then given, according to lemma, by
    # u and preimages of lower dimensional basis elements
    return u_new, u_factor, v_factor
    


In [21]:
def conv_np2sp(basis,u):
    return sp.Matrix(basis), sp.Matrix(u)

def conv_sp2np(basis):
    return np.array(basis).astype(np.float64)
    


In [56]:
def common_superlattice(basis_np,u_np):
    """
    Takes a 3x3 numpy matrix containing an integer basis,
    as well as an additional numpy vector u.
    Calculates integer basis of the superlattice spanned
    by basis and u as 3x3 numpy matrix. 
    """
    
    basis, u = conv_np2sp(basis_np, u_np)
    
    # caculate primitive of u in superlattice
    v, coeff, u_coeff = L31_parallel_vectors(basis,u)
    
    # project basis to v_orth
    Pbasis = proj_3d_basis(basis,v)
    # If vector in P(basis) is 0, then original vector is parallel to u  
    # Pick gcd of both, which should be v (?)
    for i in range(3):
        if Pbasis[:,i].norm() == 0:
            base1_3d = basis[:,(i+1)%3]
            base2_3d = basis[:,(i+2)%3]
            break
            
        elif i==2:
            # reduce to 2 component vectors and get new basis and new u
            Pbasis_2d, u_2d, u_2d_index, v_2d, v_2d_coeff = proj2induct_2d(Pbasis,v) 
            
            # project basis_2d to v_2d_orth
            PPbasis = proj_2d_basis(Pbasis_2d,v_2d)
            
            # calculate gcd of projected vectors
            x,y = gcd_of_2dvecs(PPbasis)
            
            # calculate basis of 2d space
            # base1_2d = PPbasis[:,0]*x + PPbasis[:,1]*y
            ###base2_2d, base2_factor_u, base2_factor_v = multiple_L_3_3(u_2d, v_2d)

            # calculate basis of 3d space
            base1_3d = basis[:,(u_2d_index+1)%3]*x + basis[:,(u_2d_index+2)%3]*y


            base2_3d = 0*u # <-- make this just zero vector sometime
            for i in range(3):
                base2_3d += v_2d_coeff[i] * basis[:,i]
            #base2_3d *= base2_factor_v
    
    #coeff, u_coeff
    base3_3d = coeff[0]*basis[:,0]+coeff[1]*basis[:,1]+coeff[2]*basis[:,2]+u_coeff*u
    # base3_3d=v
    ###base3_3d, base3_factor_u, base3_factor_v = multiple_L_3_3(u,v)

    new_basis = sp.Matrix([base1_3d[:], base2_3d[:], base3_3d[:]]).T
    
    #"""
    print("Final basis:")
    sp.pprint(new_basis)

    print("Original spanning set:")
    sp.pprint(basis)
    sp.pprint(u)
    #"""
    
    return conv_sp2np(new_basis)
    

### Check system of linear equations to see if old spanning set is integer combination of new basis

In [23]:
def check_reexpressibility(new_basis, old_basis, u, p=False):
    """
    Takes 3x3 numpy matrix new_basis,
          3x3 numpy matrix old_basis,
          3x1 numpy vector u.
          
    Returns the coefficients of old_basis
    and u in terms of vectors in new_basis.
    """
    r = True
    for i in range(3):  
        if p:
            print(f"Coefficient of old basis vector {i}")
        x = check_coeff(new_basis, old_basis[:,i],p)
        r = r and comp2round(x)
    
    if p:
        print("Coefficient of u")
    x = check_coeff(new_basis, u,p)
    r = r and comp2round(x)
    """
    if r:
        print("TRUE! Old basis and u can be expressed as integer combinations of the new basis.")
    else:
        print("Something went wrong, reexpression as integer combination failed.")
    """
    return r

def check_coeff(A, b, p=False):
    x = np.linalg.solve(A, b)
    if p:
        print(np.around(x,6))
    return x

def comp2round(x,eps=1e-6):
    if la.norm(x-np.around(x))<eps:
        return True
    else:
        return False

## Start example

In [57]:

for i in range(100):
    print(f"try nr. {i}")
    # generate example
    basis = gen_int_basis(a=5)
    u = gen_int_vec(a=5)

    new_basis = common_superlattice(basis,u)
    r = check_reexpressibility(new_basis, basis, u)
    if r==False:
        print("Something went wrong, reexpression as integer combination failed.")
        print(basis)
        print(u)
        break
        


try nr. 0
Final basis:
⎡6   -1  -1⎤
⎢          ⎥
⎢1   2   -1⎥
⎢          ⎥
⎣-3  -5  3 ⎦
Original spanning set:
⎡-1  4   -1⎤
⎢          ⎥
⎢2   5   2 ⎥
⎢          ⎥
⎣-5  -3  0 ⎦
⎡-1⎤
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣3 ⎦
try nr. 1
Final basis:
⎡-51  1   -1⎤
⎢           ⎥
⎢124  5   1 ⎥
⎢           ⎥
⎣114  -1  2 ⎦
Original spanning set:
⎡1   -2  -5⎤
⎢          ⎥
⎢5   -1  4 ⎥
⎢          ⎥
⎣-1  -5  -2⎦
⎡-1⎤
⎢  ⎥
⎢1 ⎥
⎢  ⎥
⎣2 ⎦
try nr. 2
Final basis:
⎡-1  47   5 ⎤
⎢           ⎥
⎢5   37   4 ⎥
⎢           ⎥
⎣2   -19  -2⎦
Original spanning set:
⎡4   -1  1 ⎤
⎢          ⎥
⎢-1  -1  -5⎥
⎢          ⎥
⎣-3  -3  -2⎦
⎡5 ⎤
⎢  ⎥
⎢4 ⎥
⎢  ⎥
⎣-2⎦
try nr. 3
Final basis:
⎡185  -3  5⎤
⎢          ⎥
⎢152  2   3⎥
⎢          ⎥
⎣54   5   0⎦
Original spanning set:
⎡-3  5  5 ⎤
⎢         ⎥
⎢2   4  5 ⎥
⎢         ⎥
⎣5   2  -3⎦
⎡5⎤
⎢ ⎥
⎢3⎥
⎢ ⎥
⎣0⎦
try nr. 4
Final basis:
⎡14   -53  3⎤
⎢           ⎥
⎢-71  -56  3⎥
⎢           ⎥
⎣-19  -72  4⎦
Original spanning set:
⎡2   -1  -3⎤
⎢          ⎥
⎢-4  -2  5 ⎥
⎢          ⎥
⎣0   2   5 ⎦
⎡3⎤
⎢ ⎥
⎢3⎥
⎢ ⎥